# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.builders import model_builder
from google.protobuf import text_format
import tensorflow as tf
from object_detection.protos import model_pb2
import time
from IPython.display import clear_output
from keras.models import load_model
import keras.backend as K
from math import floor, ceil
import cv2
import pickle as pkl

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


/home/wc-gpu/MasterThesis/models/research/object_detection/utils/visualization_utils.py:26: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

../utils/visualization_utils.py:26: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/wc

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
# MODEL_NAME = '../prod_faster_rcnn_resnet50_coco_res_512x384_iou_genration'
MODEL_NAME = '../training/keep'

# MODEL_NAME = 'faster_rcnn_resnet50_coco_2018_01_28'
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
# PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/saved_model.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
test_image_path = "/home/wc-gpu/MasterThesis/session_data/122923_testing"
# test_image_path = "/Users/mbaltac/Home/MasterThesis/MasterThesis/labeled_images_shopping_sessions/labeled/122923_testing"
TEST_IMAGE_PATHS = sorted([os.path.join(test_image_path, f) for f in os.listdir(test_image_path) if f.endswith("jpg")])


# TEST_IMAGE_PATHS = [TEST_IMAGE_PATHS[0], TEST_IMAGE_PATHS[0]]


# Size, in inches, of the output images.
IMAGE_SIZE = (18, 12)

SESSIONS_PATH = "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000"

## Download Model

In [5]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
# with tf.Graph().as_default():
#   od_graph_def = tf.GraphDef()
#   with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
#     serialized_graph = fid.read()
#     od_graph_def.ParseFromString(serialized_graph)
#     tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
# category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [8]:
category_index = {1: {'id': 1, 'name': 'product'},
 2: {'id': 2, 'name': 'add'},
 3: {'id': 3, 'name': 'remove'},
 4: {'id': 4, 'name': 'move_from'},
 5: {'id': 5, 'name': 'move_to'}}

## Helper code

In [9]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [10]:
def max_pool_dynamic_input(x, H_out, W_out):
    out = None
    (N, H, W, C) = x.shape
    pool_height = np.zeros((H_out), dtype="int32")
    pool_width = np.zeros((W_out), dtype="int32")
    
    div_h = int(H/H_out)
    mod_h = H%H_out
    div_w = int(W/W_out)
    mod_w = W%W_out
    pool_height = pool_height + div_h
    pool_width = pool_width + div_w
    pool_height[:mod_h] += 1
    pool_width[:mod_w] += 1
    
    out = np.zeros((N, H_out, W_out, C))
    for n in range(N):
        for h in range(H_out):
            for w in range(W_out):
                h1 = sum(pool_height[:h])
                h2 = h1 + pool_height[h]
                w1 = sum(pool_width[:w])
                w2 = w1 + pool_width[w]
                window = x[n, h1:h2, w1:w2, :]
                m = np.max(window, axis=0)
                m = np.max(m, axis=0)
                out[n,h,w,:] = m
    
    return out


def get_max_and_concat(*args):
    size = 0
    for arg in args:
        size += arg.shape[-1]
    final = np.zeros(size)
    
    arg_size = 0
    
    i = 0
    for arg in args:
        max_pool = max_pool_dynamic_input(np.expand_dims(arg, 0), 1, 1)[0,0,0,:]
        final[i: arg.shape[-1]+i] = max_pool
        i+= arg.shape[-1]  
    return final

def get_max_and_concat2(*args):
    size = 0
    for arg in args:
        size += arg.shape[-1]
    final = np.zeros(size)
    
    arg_size = 0

    for arg in args:
        for i in range(arg.shape[-1]):
            final[i+arg_size] = np.max(arg[:,:,i])
        arg_size += arg.shape[-1]
    
#     mn = np.mean(final)
#     std = np.std(final)
#     final = (final - mn)/std

    return final

def max_pooled(previous_features, features, label):
    box_prev = previous_features[
        floor(label[0]*previous_features.shape[0]):
        ceil(label[2]*previous_features.shape[0]), 
        floor(label[1]*previous_features.shape[1]):
        ceil(label[3]*previous_features.shape[1]),:]
    box_cur = features[
        floor(label[0]*features.shape[0]):
        ceil(label[2]*features.shape[0]), 
        floor(label[1]*features.shape[1]):
        ceil(label[3]*features.shape[1]),:]

    return get_max_and_concat(box_prev, previous_features, box_cur, features)

In [11]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    """

    b1y1, b1x1, b1y2, b1x2 = bb1
    b2y1, b2x1, b2y2, b2x2 = bb2

    b1y1, b1x1, b1y2, b1x2 = b1y1, b1x1, b1y1+b1y2, b1x1+b1x2
    b2y1, b2x1, b2y2, b2x2 = b2y1, b2x1, b2y1+b2y2, b2x1+b2x2

    b1y1, b1x1, b1y2, b1x2 = b1x1, b1y1, b1x2, b1y2
    b2y1, b2x1, b2y2, b2x2 = b2x1, b2y1, b2x2, b2y2
    
    # determine the coordinates of the intersection rectangle
    x_left = max(b1x1, b2x1)
    y_top = max(b1y1, b2y1)
    x_right = min(b1x2, b2x2)
    y_bottom = min(b1y2, b2y2)

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (b1x2 - b1x1) * (b1y2 - b1y1)
    bb2_area = (b2x2 - b2x1) * (b2y2 - b2y1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou > 0.0
    assert iou < 1.0
    return iou


def filter_detections(out_boxes, out_scores, iou_tresh = .5, score_tresh = 0.3):
    fboxes, fscores, fclasses = [], [], []
    removed = set()
    
    filter_score = np.where(out_scores > score_tresh)
    print (filter_score)
    
    for i, b in reversed(list(enumerate(out_boxes))):
        if i not in removed:
            max_score = out_scores[i]
            max_idx = i
            for j in range(i+1, len(out_boxes)):
                if j not in removed and get_iou(b, out_boxes[j]) > .65:
                    if out_scores[j] > max_score:
                        removed.add(max_idx)
                        max_idx = j
                        max_score = out_scores[j]
                    else:
                        removed.add(j)

    for i in range(len(out_boxes)):
        if i not in removed:
            fboxes.append(out_boxes[i])
            fscores.append(out_scores[i])
            fclasses.append(out_classes[i])

    return fboxes, fscores, fclasses

In [12]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.9

In [13]:
def batch_gen(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i: min(i + n, len(l))]
        
def run_inference_for_batch(images, batch = 16):
    with tf.Session(config=tf_config) as sess:
#         batches = batch_gen(images, batch)
      # Get handles to input and output tensors
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
        ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                  tensor_name)
        features = tf.get_default_graph().get_tensor_by_name(
            'FirstStageFeatureExtractor/resnet_v1_50/resnet_v1_50/block3/unit_6/bottleneck_v1/Relu:0')

        tensor_dict['features'] = tf.expand_dims(features, 0)
        
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
        
        # Run inference
        output_dict = sess.run(tensor_dict, 
                             feed_dict={image_tensor: images})

        # all outputs are float32 numpy arrays, so convert types as appropriate
        output_dict['num_detections'] = output_dict['num_detections'].astype(np.uint8)
        output_dict['detection_classes'] = output_dict[
          'detection_classes'].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes']
        output_dict['detection_scores'] = output_dict['detection_scores']
        output_dict['features'] = output_dict['features'][0]

        return output_dict

In [20]:
def read_images(session_path):
    image_paths = sorted([f for f in os.listdir(session_path) if f.endswith(".jpg")])
    prev_features = None
    
    
    images = []
    for image_fname in image_paths:
        t0 = time.time()
        image_path = os.path.join(session_path, image_fname)

        image = Image.open(image_path)
        image_np = load_image_into_numpy_array(image)
        images.append(image_np)
        print ("reading images", str(len(images)/len(image_paths))[:5], end="\r")
    
    return images, image_paths


def process_images_batch(images, batch_size = 16, total_images=None):
    progress = 0
    processed = 0
    time_avg = 0

    outputs = []
    total_images = len(images)

    total_time = 0
    box_predictions = []
    tf.reset_default_graph()
    with tf.Graph().as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        image_batch = batch_gen(images, batch_size)
        for img_b in image_batch:
            t0 = time.time()
            output_dict = run_inference_for_batch(img_b, batch_size)
            outputs.append(output_dict)

            progress += 1
            processed += 1
            t1 = time.time()

            total = t1-t0

            time_avg = (total_time + total) / processed

            total_time += total

            log_text = " ".join(["progress ", ":", str(progress / total_images*batch_size), 
                                 str(progress), str(total_images)])
            log_text += " | "

            log_text += "".join(["process time ", str(total)])
            log_text += " | "
            log_text += "".join([" time passed: ", str(time_avg*processed) ," s, remaining: ", str(time_avg* (total_images/batch_size - progress) ), " s"])
        

            print (log_text, end="                                       \r")

    return outputs
    
def process_session_get_outputs(session_path, batch_size=16, total_images=None):
    output_splitted = []
    images, image_paths = read_images(session_path)
    outputs_batch  = process_images_batch(images, total_images=None)
    
    i=0
    for big_dict in outputs_batch:
        for j in range(len(big_dict["num_detections"])):
            output_dict = {}
            for k in big_dict.keys():
                output_dict[k] = big_dict[k][j]
            output_dict["image_path"] = image_paths[i*batch_size + j]
            output_dict["image_np"] = images[i*batch_size + j]
            output_splitted.append(output_dict)
        i += 1
    
    return output_splitted, images, image_paths

In [15]:
out_path = "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000/processed_sessions"

In [16]:
visualize_debug = False

features = None
total_images = 0
sessions = os.listdir(SESSIONS_PATH)
actual_sessions = []

for session in sorted(sessions, reverse=True):
    try:
        a = int(session)
        session_path = os.path.join(SESSIONS_PATH, session)
        total_images += len([f for f in os.listdir(session_path) if f.endswith(".jpg")])
        actual_sessions.append(session)
    except:
        continue


In [ ]:
outputs = []
count = 0
log= open("feature_extraction_new.log","w+")
for session in sorted(actual_sessions, reverse=True):
    session_path = os.path.join(SESSIONS_PATH, session)
    images = [f for f in os.listdir(session_path) if f.endswith(".jpg")]
    print(session, str(count/len(actual_sessions))[:6], "                                        ")   
    count += 1
    try:
        session_output_path = os.path.join(out_path, session)
        if not os.path.exists(session_output_path):
            outputs = process_session_get_outputs(session_path)
            with open(session_output_path, "wb") as f:
                pkl.dump(outputs, f)
    except Exception as ex:
        print ("IMAGE ", image_fname)
        print (str(ex))

164413 0.0                                         
164398 0.0020                                         
164384 0.0041                                         80115842819214 |  time passed: 11.084317922592163 s, remaining: -1.3855397403240204 s                                       
164378 0.0062                                         1518611907959 |  time passed: 15.672352313995361 s, remaining: -2.285551379124324 s                                         
164377 0.0082                                         387222290039062 |  time passed: 9.138337135314941 s, remaining: -0.5711460709571838 s                                       
164375 0.0103                                         3 |  time passed: 9.720974445343018 s, remaining: -2.126463159918785 s                                       
164372 0.0124                                         95432138442993 |  time passed: 7.856424570083618 s, remaining: -0.49102653563022614 s                                       
164349 0.0145

164020 0.0975                                         46076774597168 |  time passed: 7.798190116882324 s, remaining: -0.1624622941017151 s                                       
164017 0.0995                                         5765061378479 |  time passed: 9.579110622406006 s, remaining: -2.2451040521264076 s                                       
164004 0.1016                                         905492782592773 |  time passed: 12.513403415679932 s, remaining: -2.189845597743988 s                                       
164002 0.1037                                         46910810470581 |  time passed: 10.24669599533081 s, remaining: -1.2808369994163513 s                                        
163995 0.1058                                         938615798950195 |  time passed: 9.951313257217407 s, remaining: -1.5548926964402199 s                                       
163989 0.1078                                         11462688446045 |  time passed: 11.860358476638794 s, r

162766 0.1908                                         884931564331055 |  time passed: 7.149441242218018 s, remaining: -0.2978933850924174 s                                       
162761 0.1929                                         725065231323242 |  time passed: 15.165385961532593 s, remaining: -1.895673245191574 s                                       
162758 0.1950                                         0496177673339844 |  time passed: 22.92380452156067 s, remaining: -1.7511239565081067 s                                        
162746 0.1970                                         77023506164551 |  time passed: 12.628536939620972 s, remaining: -1.7364238291978835 s                                       
162744 0.1991                                         891581058502197 |  time passed: 10.17249059677124 s, remaining: -1.1126161590218544 s                                       
162743 0.2012                                         412936210632324 |  time passed: 12.93677687644958

In [ ]:
score_tresh = .6

In [ ]:
def filter_scores(output_dict):
    filter_scores_idx = np.where(output_dict['detection_scores'] > score_tresh)[0]
    for k in output_dict.keys():
        try:
            output_dict[k] = output_dict[k][filter_scores_idx]
        except:
            continue

In [ ]:
output_dict["image_np"] = image_np
output_dict["detection_scores"] = np.random.uniform(low=0.7, high=1, size=output_dict["detection_scores"].shape)
print (output_dict["detection_scores"])


In [ ]:
from IPython.display import display, clear_output
clear_output(wait=True)

i=0
for output_dict in output_splitted:
    not_same = np.where(output_dict["detection_classes"] > 0)
    image_np = output_dict["image_np"]
    image_np_out = np.copy(image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
          image_np_out,
          output_dict['detection_boxes'][not_same],
          output_dict["detection_classes"][not_same],
          output_dict['detection_scores'][not_same],
          category_index,
          use_normalized_coordinates=True,
          line_thickness=5,
          min_score_thresh=.5)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np_out)


In [ ]:
norm_image = lambda image: cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)


In [ ]:
prev = None
for output_dict in output_splitted:
    image = output_dict["image_np"]
    if prev is not None:
        dif = norm_image(image)-norm_image(prev)
        plt.figure(figsize=IMAGE_SIZE)
        plt.imshow(norm_image(dif))
    prev = image

In [ ]:
# len(output_splitted)

In [ ]:
# ker_model = load_model(
#     "/home/wc-gpu/MasterThesis/models/research/object_detection/box_classification_checkpoint/keep/2019-03-26weights-epoch59-val_acc0.84-val_loss1.08_add_1prev_sessions.hdf5")

In [ ]:
# mn, std = np.load("./mn_std.pkl")

In [ ]:
# def cos(vA, vB):
#     return np.dot(vA, vB) / (np.sqrt(np.dot(vA,vA)) * np.sqrt(np.dot(vB,vB)))

In [ ]:
# def sift    
#     sift = cv2.xfeatures2d.SIFT_create()
#     kp_1, desc_1 = sift.detectAndCompute(original, None)
#     kp_2, desc_2 = sift.detectAndCompute(image_to_compare, None)
#     index_params = dict(algorithm=0, trees=5)
#     search_params = dict()
#     flann = cv2.FlannBasedMatcher(index_params, search_params)
#     matches = flann.knnMatch(desc_1, desc_2, k=2)
#     good_points = []
#     for m, n in matches:
#         if m.distance < 0.6*n.distance:
#             good_points.append(m)
#     # Define how similar they are
#     number_keypoints = 0
#     if len(kp_1) <= len(kp_2):
#         number_keypoints = len(kp_1)
#     else:
#         number_keypoints = len(kp_2)


In [ ]:
# prev_features = []
# for output_dict in output_splitted:
#     print 
#     if len(prev_features):
#         box_features = []
#         for b in output_dict['detection_boxes']:
# #             print(prev_features.shape, output_dict['features'].shape, b)
#             box_features.append(max_pooled(prev_features,  output_dict['features'], b))

#         if len(box_features):
#             box_features = np.array(box_features)

#             box_features = (box_features - mn)/std
# #             box_prediction = ker_model.predict(box_features)
#             box_prediction = np.zeros(len(box_features))
            
#             for i in range(len(box_features)):
#                 box_prediction[i] = np.sum(np.absolute(box_features[i][:2048] - box_features[i][2048:]))
                
#         #                 classes = np.argmax(box_prediction, 1)
#         #                 scores = []
#         #                 for i, s in enumerate(box_prediction):
#         #                     scores.append(s[classes[i]])                
#         #                 scores = np.array(scores)
#             output_dict['detection_scores'] *= box_prediction.flatten()
#         #                 output_dict['detection_classes'] = classes + 1

#     prev_features = output_dict['features']